# Métricas de evaluación

## Configuración

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

## Similitud de texto

### Evaluación con múltiples métricas de similitud de texto usando el dataset de Quora

#### Explicación breve

**Flujo Completo del Código**

1. Dataset:
Descargamos y cargamos el dataset de Quora, obteniendo las consultas y el corpus.

2. Embeddings:
Usamos Sentence Transformers para convertir las consultas y documentos en vectores densos.

3. Similitud:
Calculamos la similitud coseno entre consultas y documentos.
Extraemos los documentos más relevantes para cada consulta.

4. Resultado:
Mostramos los pares de consulta-documento con mayor similitud.

**Ejemplo de Salida**

***Supongamos que la consulta es:***

Consulta: How can I learn Python effectively?

***Documentos más similares:***

1. DocID: 12345, Puntaje: 0.87, Texto: What are the best resources for learning Python?
2. DocID: 67890, Puntaje: 0.82, Texto: How do I start with Python programming?
3. DocID: 13579, Puntaje: 0.78, Texto: What are tips to become proficient in Python?

In [2]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader

# Descarga el dataset Quora
dataset_url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/quora.zip"
data_path = util.download_and_unzip(dataset_url, "datasets/")  # Descarga y descomprime

# Cargar el dataset
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")  # Usa el split "test"


/opt/miniconda3/envs/nlp_tutorials/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/522931 [00:00<?, ?it/s]

**¿Qué es Sentence Transformers?**

* Sentence Transformers es una biblioteca que utiliza modelos basados en Transformers (como BERT o RoBERTa) para generar embeddings de alta calidad.

* Estos embeddings capturan la semántica del texto, lo que permite comparar textos mediante operaciones matemáticas como la Similitud Coseno.

¿Por qué usamos all-MiniLM-L6-v2?

1. Modelo Ligero:
* Es un modelo pequeño y eficiente, diseñado para generar embeddings densos rápidamente.
* MiniLM utiliza una arquitectura reducida en comparación con modelos más grandes como BERT o RoBERTa, lo que lo hace más rápido y menos exigente en términos de recursos.

2. Versatilidad:
* Funciona bien en tareas generales de similitud de texto, como búsqueda de documentos o detección de duplicados.
* Está optimizado para encontrar similitudes semánticas entre oraciones o párrafos.

3. Calidad vs. Eficiencia:
* Aunque es más pequeño que otros modelos, mantiene una excelente relación calidad/eficiencia.
* Es ideal para aplicaciones prácticas en tiempo real o con grandes volúmenes de datos.

4. Preentrenamiento:
* Fue preentrenado específicamente para tareas de comparación de oraciones y recuperación de información.
* Es compatible con métricas como Similitud Coseno y Distancia Euclidiana.

**¿Qué hacen los embeddings generados por este modelo?**

Transforman textos en vectores densos. Por ejemplo, el texto "¿Cómo aprender Python rápidamente?" se convierte en un vector numérico.

* Los vectores se pueden comparar para medir la similitud semántica entre textos.
* Los embeddings permiten realizar búsquedas eficientes, detectar duplicados o clasificar textos relacionados.


In [3]:
from sentence_transformers import SentenceTransformer

# Cargar modelo de Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

# Seleccionar 50 consultas
subset_queries = list(queries.values())[:50]  # Ajusta el número según lo que necesites
subset_corpus = list(corpus.values())[:1000]  # También puedes reducir el corpus

# Generar embeddings solo para el subconjunto
query_embeddings = model.encode(subset_queries, convert_to_tensor=True)
corpus_embeddings = model.encode(subset_corpus, convert_to_tensor=True)

print(f"Embeddings generados para {len(subset_queries)} consultas y {len(subset_corpus)} documentos.")


Embeddings generados para 50 consultas y 1000 documentos.


#### Similitud del Coseno

In [4]:
from torch import topk
from sentence_transformers.util import cos_sim

# Calculate cosine similarity between queries and documents
similarity_scores_cosine = cos_sim(query_embeddings, corpus_embeddings)

# Get the most similar documents for the first query
top_k = 5
scores_cosine_p1, indices_cosine_p1 = topk(similarity_scores_cosine[0], k=top_k)

# Convert tensor to NumPy arrays
scores_cosine_p1 = scores_cosine_p1.cpu().numpy()
indices_cosine_p1 = indices_cosine_p1.cpu().numpy()

# Retrieve the first query's content
query_content = list(queries.values())[0]

# Retrieve the content of the most similar documents
similar_docs = [list(corpus.values())[idx] for idx in indices_cosine_p1]

# Display the query and the most similar documents
print(f"Query: {query_content}\n")
print("Most Similar Documents:")
for i, (doc, score) in enumerate(zip(similar_docs, scores_cosine_p1), start=1):
    print(f"{i}. Similarity Score: {score:.4f}")
    print(f"   Document Content: {doc}")
    print()

Query: Which question should I ask on Quora?

Most Similar Documents:
1. Similarity Score: 0.6616
   Document Content: {'text': 'What are the questions should not ask on Quora?', 'title': ''}

2. Similarity Score: 0.4923
   Document Content: {'text': 'How do you answer a question on Quora?', 'title': ''}

3. Similarity Score: 0.4099
   Document Content: {'text': "What is the best thing you've ever learned on Quora?", 'title': ''}

4. Similarity Score: 0.4046
   Document Content: {'text': 'What should I ask my crush?', 'title': ''}

5. Similarity Score: 0.3705
   Document Content: {'text': 'Why nobody answer my questions in Quora?', 'title': ''}



#### Distancia euclidiana

In [5]:
from torch import cdist
import numpy as np

# Create a subset of embeddings (first 50 queries and first 1000 documents)
subset_queries = query_embeddings[:50]  # Adjust as needed
subset_corpus = corpus_embeddings[:1000]  # Adjust as needed

# Compute Euclidean distances between queries and the corpus
distances_euclidean = cdist(subset_queries, subset_corpus, p=2).cpu().numpy()
print("\nDistancias Euclidianas calculadas con éxito.")

# Select the 5 closest documents for the first query
top_k = 5
distances_euclidean_p1 = distances_euclidean[0]
indices_euclidean_p1 = np.argsort(distances_euclidean_p1)[:top_k]  # Indices of the smallest distances
scores_euclidean_p1 = distances_euclidean_p1[indices_euclidean_p1]

# Retrieve the first query's content
query_content = list(queries.values())[0]

# Retrieve the content of the most similar documents
similar_docs = [list(corpus.values())[idx] for idx in indices_euclidean_p1]

# Display the query and the most similar documents
print(f"Query: {query_content}\n")
print("Most Similar Documents (Euclidean Distance):")
for i, (doc, score) in enumerate(zip(similar_docs, scores_euclidean_p1), start=1):
    print(f"{i}. Distance: {score:.4f}")
    print(f"   Document Content: {doc}")
    print()



Distancias Euclidianas calculadas con éxito.
Query: Which question should I ask on Quora?

Most Similar Documents (Euclidean Distance):
1. Distance: 0.8227
   Document Content: {'text': 'What are the questions should not ask on Quora?', 'title': ''}

2. Distance: 1.0077
   Document Content: {'text': 'How do you answer a question on Quora?', 'title': ''}

3. Distance: 1.0863
   Document Content: {'text': "What is the best thing you've ever learned on Quora?", 'title': ''}

4. Distance: 1.0912
   Document Content: {'text': 'What should I ask my crush?', 'title': ''}

5. Distance: 1.1221
   Document Content: {'text': 'Why nobody answer my questions in Quora?', 'title': ''}



#### Similitud de Jaccard

In [6]:
# Definir la función de distancia de Jaccard
def jaccard_distance(text1, text2):
    set1, set2 = set(text1.split()), set(text2.split())
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return 1 - (intersection / union) if union != 0 else 1

In [7]:
# Define Jaccard Distance function
def jaccard_distance(text1, text2):
    set1, set2 = set(text1.split()), set(text2.split())
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return 1 - (intersection / union) if union != 0 else 1

# Select a subset of data (50 queries and 1000 documents)
top_k = 5
subset_queries = [q['text'] if isinstance(q, dict) else q for q in list(queries.values())[:50]]
subset_corpus = [doc['text'] if isinstance(doc, dict) else doc for doc in list(corpus.values())[:1000]]

# Calculate Jaccard distances between the first query and all documents
jaccard_distances_question_p1 = [jaccard_distance(subset_queries[0], doc) for doc in subset_corpus]
indices_jaccard_p1 = np.argsort(jaccard_distances_question_p1)[:top_k]  # Indices of the smallest distances
scores_jaccard_p1 = [jaccard_distances_question_p1[i] for i in indices_jaccard_p1]

# Retrieve the content of the most similar documents
query_content = subset_queries[0]
similar_docs = [subset_corpus[idx] for idx in indices_jaccard_p1]

# Display the query and the most similar documents
print(f"Query: {query_content}\n")
print("Most Similar Documents (Jaccard Distance):")
for i, (doc, score) in enumerate(zip(similar_docs, scores_jaccard_p1), start=1):
    print(f"{i}. Distance: {score:.4f}")
    print(f"   Document Content: {doc}")
    print()


Query: Which question should I ask on Quora?

Most Similar Documents (Jaccard Distance):
1. Distance: 0.6667
   Document Content: What are the questions should not ask on Quora?

2. Distance: 0.7000
   Document Content: What should I ask my crush?

3. Distance: 0.7000
   Document Content: Can I earn money on Quora?

4. Distance: 0.7500
   Document Content: How do you answer a question on Quora?

5. Distance: 0.7778
   Document Content: How should I study



#### Comparamos los 3 resultados

* Notamos que la Euclidanea con la de similitud del coseno ordenan de forma similar pero la métrica del coseno captura mejor la similitud al diferenciar más entre preguntas como estas ***What should I ask my crush?*** con ***How do you answer a question on Quora?***

* Se muestra que la distancia de jaccard no sirve cuando quieres establecer relaciones semánticas

In [8]:
# Define the top_k for consistency
top_k = 5

# Jaccard
jaccard_distances_question_p1 = [jaccard_distance(subset_queries[0], doc) for doc in subset_corpus]
indices_jaccard_p1 = np.argsort(jaccard_distances_question_p1)[:top_k]
scores_jaccard_p1 = [jaccard_distances_question_p1[i] for i in indices_jaccard_p1]

# Euclidean
distances_euclidean_p1 = distances_euclidean[0]
indices_euclidean_p1 = np.argsort(distances_euclidean_p1)[:top_k]
scores_euclidean_p1 = [distances_euclidean_p1[i] for i in indices_euclidean_p1]

# Cosine
cosine_similarities_question_1 = similarity_scores_cosine[0].cpu().numpy()
indices_cosine_p1 = np.argsort(-cosine_similarities_question_1)[:top_k]
scores_cosine_p1 = [cosine_similarities_question_1[i] for i in indices_cosine_p1]

# Consolidate all document indices that appeared in any top 5
all_indices = set(indices_jaccard_p1) | set(indices_euclidean_p1) | set(indices_cosine_p1)

# Create the DataFrame
matrix_data = {
    "Document Content": [subset_corpus[idx] for idx in all_indices],
    "Jaccard": [
        scores_jaccard_p1[indices_jaccard_p1.tolist().index(idx)] if idx in indices_jaccard_p1 else np.nan
        for idx in all_indices
    ],
    "Euclidean": [
        scores_euclidean_p1[indices_euclidean_p1.tolist().index(idx)] if idx in indices_euclidean_p1 else np.nan
        for idx in all_indices
    ],
    "Cosine": [
        scores_cosine_p1[indices_cosine_p1.tolist().index(idx)] if idx in indices_cosine_p1 else np.nan
        for idx in all_indices
    ],
}

df_matrix = pd.DataFrame(matrix_data)

# Display the matrix
print("\nMatrix of Document Content and Metric Values:")
df_matrix



Matrix of Document Content and Metric Values:


,Document Content,Jaccard,Euclidean,Cosine
0,What are the questions should not ask on Quora?,0.666667,0.822656,0.661619
1,Can I earn money on Quora?,0.700000,NaN,NaN
2,How should I study,0.777778,NaN,NaN
3,How do you answer a question on Quora?,0.750000,1.007705,0.492265
4,Why nobody answer my questions in Quora?,NaN,1.122088,0.370459
5,What should I ask my crush?,0.700000,1.091194,0.404648
6,What is the best thing you've ever learned on ...,NaN,1.086328,0.409946
